## Setup

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop
from kepler.pandas.decorators import RingerDecorator
from itertools import product
import rootplotlib as rpl
import mplhep as hep
import root_numpy
import ROOT
ROOT.gStyle.SetOptStat(0);
import numpy as np
import pandas as pd
import glob
import os
import logging
import matplotlib.pyplot as plt
plt.style.use(hep.style.ROOT)
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from packages.generators import ring_percentages, RingGenerator
from packages.plotting import make_plot_fig
from packages.utils import get_logger

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence


In [2]:
drop_cols = drop_columns = [
                    'RunNumber', 
                    'trig_L2_cl_e2tsts1',
                    'trig_L2_el_hastrack',
                    'trig_L2_el_eta',
                    'trig_L2_el_phi',
                    'trig_L2_el_caloEta',
                    'trig_L2_el_trkClusDeta',
                    'trig_L2_el_trkClusDphi',
                    'trig_L2_el_etOverPt',
                    'trig_EF_cl_hascluster',
                    'trig_EF_cl_eta',
                    'trig_EF_cl_etaBE2',
                    'trig_EF_cl_phi',     
                    'trig_EF_el_hascand',
                    'trig_EF_el_eta',
                    'trig_EF_el_etaBE2',
                    'trig_EF_el_phi',
                    'trig_EF_el_rhad1',
                    'trig_EF_el_rhad',
                    'trig_EF_el_f3',
                    'trig_EF_el_weta2',
                    'trig_EF_el_rphi',
                    'trig_EF_el_reta',
                    'trig_EF_el_wtots1',
                    'trig_EF_el_eratio',
                    'trig_EF_el_f1',
                    'trig_EF_el_hastrack',
                    'trig_EF_el_deltaEta1',
                    'trig_EF_el_deltaPhi2',
                    'trig_EF_el_deltaPhi2Rescaled',
                    'el_etaBE2',
                    'el_numberOfBLayerHits',
                    'el_numberOfPixelHits',
                    'el_numberOfTRTHits',
                    'el_trans_TRT_PID',
                    'el_deltaPhi2',
                    'el_TaP_Mass',
                ]

l1seeds_per_energy = {
    24: 'L1_EM22VHI',
    26: 'L1_EM22VHI',
    60: 'L1_EM24VHI',
    140: 'L1_EM24VHI'
}

criteria_conf_names = {
    'tight': 'ElectronRingerTightTriggerConfig.conf',
    'medium': 'ElectronRingerMediumTriggerConfig.conf',
    'loose': 'ElectronRingerLooseTriggerConfig.conf',
    'vloose': 'ElectronRingerVeryLooseTriggerConfig.conf'
}

energy_chains = {
     24: 'e24_lhtight_nod0_{strategy}_ivarloose',
     26: 'e26_lhtight_nod0_{strategy}_ivarloose',
     60: 'e60_lhmedium_nod0_{strategy}_L1EM24VHI',
     140: 'e140_lhloose_nod0_{strategy}'
}

steps_choices = ['L2Calo', 'L2', 'EFCalo', 'HLT']

## Parameters

In [3]:
datasetpath = '/home/lbarranunes/data/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins'

modelpaths = [
    'tunings/TrigL2_20180125_v8',
    'tunings/TrigL2_20210907_v12 (v8.1-2)',
    'tunings/TrigL2_20220704_v20',
    'tunings/TrigL2_20220802_v20.1'
    # 'tunings/TrigL2_20210306_v9'
]

output_dir = 'analysis/2022_08_17_v20.1'
cutbased = False
plot_vars = ['et', 'eta', 'pt', 'mu']
values = ['pd', 'fr']

chain_names = [
    'e24_lhtight_nod0_{strategy}_ivarloose',
    'e26_lhtight_nod0_{strategy}_ivarloose',
    'e60_lhmedium_nod0_{strategy}_L1EM24VHI',
    'e140_lhloose_nod0_{strategy}'
]

trigger_steps = ['L2Calo', 'L2', 'EFCalo', 'HLT']
dev = True
log = False

# Initializing logger
analysis_logger = get_logger('run_analysis', file=log)


## Building Decorators

In [4]:
analysis_logger.info('Building decorators')
decorators = list()
trigger_strategies = ['noringer'] if cutbased else list()
for modelpath, criterion in product(modelpaths, criteria_conf_names.keys()):
    conf_name = criteria_conf_names[criterion]
    confpath = os.path.join(modelpath, conf_name)
    env = ROOT.TEnv(confpath)
    ringer_version = env.GetValue("__version__", '')
    ringer_name = f'ringer_{ringer_version}'
    analysis_logger.info(f'Building decorator for {confpath}. Version: {ringer_version}')
    decorator = RingerDecorator(f'{ringer_name}_{criterion}', confpath, RingGenerator(ring_percentages[ringer_version]))
    decorators.append(decorator)
    if not ringer_name in trigger_strategies:
        trigger_strategies.append(ringer_name)

2022-08-17 21:40:45,970 - Building decorators
2022-08-17 21:40:45,987 - Building decorator for tunings/TrigL2_20180125_v8/ElectronRingerTightTriggerConfig.conf. Version: v8
2022-08-17 21:40:46,761 - Building decorator for tunings/TrigL2_20180125_v8/ElectronRingerMediumTriggerConfig.conf. Version: v8
2022-08-17 21:40:47,207 - Building decorator for tunings/TrigL2_20180125_v8/ElectronRingerLooseTriggerConfig.conf. Version: v8
2022-08-17 21:40:47,611 - Building decorator for tunings/TrigL2_20180125_v8/ElectronRingerVeryLooseTriggerConfig.conf. Version: v8
2022-08-17 21:40:48,010 - Building decorator for tunings/TrigL2_20210907_v12 (v8.1-2)/ElectronRingerTightTriggerConfig.conf. Version: v12
2022-08-17 21:40:48,426 - Building decorator for tunings/TrigL2_20210907_v12 (v8.1-2)/ElectronRingerMediumTriggerConfig.conf. Version: v12
2022-08-17 21:40:48,818 - Building decorator for tunings/TrigL2_20210907_v12 (v8.1-2)/ElectronRingerLooseTriggerConfig.conf. Version: v12
2022-08-17 21:40:49,310 - 

2022-08-17 21:40:45,989 | Py.RingerDecorator                      INFO Reading... tunings/TrigL2_20180125_v8/ElectronRingerTightTriggerConfig.conf
2022-08-17 21:40:46,763 | Py.RingerDecorator                      INFO Reading... tunings/TrigL2_20180125_v8/ElectronRingerMediumTriggerConfig.conf
2022-08-17 21:40:47,208 | Py.RingerDecorator                      INFO Reading... tunings/TrigL2_20180125_v8/ElectronRingerLooseTriggerConfig.conf
2022-08-17 21:40:47,613 | Py.RingerDecorator                      INFO Reading... tunings/TrigL2_20180125_v8/ElectronRingerVeryLooseTriggerConfig.conf
2022-08-17 21:40:48,011 | Py.RingerDecorator                      INFO Reading... tunings/TrigL2_20210907_v12 (v8.1-2)/ElectronRingerTightTriggerConfig.conf
2022-08-17 21:40:48,428 | Py.RingerDecorator                      INFO Reading... tunings/TrigL2_20210907_v12 (v8.1-2)/ElectronRingerMediumTriggerConfig.conf
2022-08-17 21:40:48,819 | Py.RingerDecorator                      INFO Reading... tunings/Tr

2022-08-17 21:40:46.011839: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-08-17 21:40:46.011891: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: UNKNOWN ERROR (-1)
2022-08-17 21:40:46.011917: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (caloba52): /proc/driver/nvidia/version does not exist
2022-08-17 21:40:46.012246: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-17 21:40:46.051319: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2400000000 Hz
2022-08-17 21:40:46.060015: I tensorflow/compiler/xla/service/service.c

## Building Chains

In [5]:
analysis_logger.info('Building chains')
chains = list()
step_chain_names = list()
for chain_name, strategy in product(chain_names, trigger_strategies):
    spliited_chain_name = chain_name.split('_')
    criterion = spliited_chain_name[1].replace('lh', '')
    step_chain_name = f'HLT_{chain_name.format(strategy=strategy)}'
    step_chain_names.append(step_chain_name)
    energy = int(spliited_chain_name[0][1:])
    l1seed = l1seeds_per_energy[energy]
    l2calo_column = f'{strategy}_{criterion}'
    analysis_logger.info(f'Building chain: {step_chain_name} model: {l2calo_column}')
    if strategy == 'noringer':
        chain = Chain(step_chain_name, L1Seed=l1seed)
    else:
        chain = Chain(step_chain_name, L1Seed=l1seed, l2calo_column=l2calo_column)
    chains.append(chain)

2022-08-17 21:40:56,936 - Building chains
2022-08-17 21:40:56,938 - Building chain: HLT_e24_lhtight_nod0_ringer_v8_ivarloose model: ringer_v8_tight
2022-08-17 21:40:56,939 - Building chain: HLT_e24_lhtight_nod0_ringer_v12_ivarloose model: ringer_v12_tight
2022-08-17 21:40:56,941 - Building chain: HLT_e24_lhtight_nod0_ringer_v20_ivarloose model: ringer_v20_tight
2022-08-17 21:40:56,942 - Building chain: HLT_e24_lhtight_nod0_ringer_v20.1_ivarloose model: ringer_v20.1_tight
2022-08-17 21:40:56,943 - Building chain: HLT_e26_lhtight_nod0_ringer_v8_ivarloose model: ringer_v8_tight
2022-08-17 21:40:56,950 - Building chain: HLT_e26_lhtight_nod0_ringer_v12_ivarloose model: ringer_v12_tight
2022-08-17 21:40:56,951 - Building chain: HLT_e26_lhtight_nod0_ringer_v20_ivarloose model: ringer_v20_tight
2022-08-17 21:40:56,952 - Building chain: HLT_e26_lhtight_nod0_ringer_v20.1_ivarloose model: ringer_v20.1_tight
2022-08-17 21:40:56,953 - Building chain: HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI model:

OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (21000.0, 'ringer_v8_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 24000.0),
             ('HLT', (24000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (21000.0, 'ringer_v12_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 24000.0),
             ('HLT', (24000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (21000.0, 'ringer_v20_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 24000.0),
             ('HLT', (24000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (21000.0, 'ringer_v20.1_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 24000.0),
             ('HLT', (24000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1

## Reading Files

In [6]:
analysis_logger.info('Loading the data')
filename_end = '*et4_eta4.npz' if dev else '*.npz'    #If dev, loads leblon
datafiles = glob.glob(os.path.join(datasetpath, filename_end))  
analysis_logger.info(f'glob_path: {os.path.join(datasetpath, filename_end)}')
data = load_in_loop(datafiles, drop_columns=drop_cols, decorators=decorators, chains=chains)

2022-08-17 21:40:56,980 - Loading the data
Reading files...:   0%|          | 0/1 [00:00<?, ?it/s]

L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v12_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v20_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v20.1_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v12_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v20_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v20.1_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM24VHI ringer_v8_medium trig_L2_el_cut_pt50toInf trig_EF_el_lhmedium
L1_EM24VHI ringer_v12_medium trig_L2_el_cut_pt50toInf trig_EF_el_lhmedium
L1_EM24VHI ringer_v20_medium trig_L2_el_cut_pt50toInf trig_EF_el_lhmedium
L1_EM24VHI ringer_v20.1_medium trig_L2_el_cut_pt50toInf trig_EF_el_lhmedium
L1_EM24VHI ringer_v8_loose trig_L2_el_cut_pt50toInf t

Reading files...: 100%|██████████| 1/1 [00:22<00:00, 22.38s/it]

L1_EM24VHI ringer_v12_loose trig_L2_el_cut_pt50toInf trig_EF_el_lhloose
L1_EM24VHI ringer_v20_loose trig_L2_el_cut_pt50toInf trig_EF_el_lhloose
L1_EM24VHI ringer_v20.1_loose trig_L2_el_cut_pt50toInf trig_EF_el_lhloose


## Plotting

In [7]:
analysis_logger.info('Making plots')
for value, var, chain_name, step in product(values, plot_vars, chain_names, trigger_steps):
    plot_dir = os.path.join(output_dir, value, var)
    analysis_logger.info(f'Plotting value: {value}, step: {step}, chain_name: {chain_name}, var: {var}')
    make_plot_fig(data, step, chain_name, trigger_strategies, plot_dir , var, value, joblib_dump=True)

2022-08-17 21:41:19,544 - Making plots
2022-08-17 21:41:19,546 - Plotting value: pd, step: L2Calo, chain_name: e24_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-17 21:41:20,211 - Plotting value: pd, step: L2, chain_name: e24_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-17 21:41:20,618 - Plotting value: pd, step: EFCalo, chain_name: e24_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-17 21:41:20,999 - Plotting value: pd, step: HLT, chain_name: e24_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-17 21:41:21,357 - Plotting value: pd, step: L2Calo, chain_name: e26_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-17 21:41:21,756 - Plotting value: pd, step: L2, chain_name: e26_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-17 21:41:22,146 - Plotting value: pd, step: EFCalo, chain_name: e26_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-17 21:41:22,527 - Plotting value: pd, step: HLT, chain_name: e26_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-17 21:41:22,888